In [1]:
import pandas as pd


In [2]:
df = pd.read_parquet('../../data/processed/challenge_campus_biomedico_2024_cleaned.parquet')

In [3]:
# List of tuples containing the code-description column pairs to be compared.
columns_pairs = [
    ('codice_provincia_residenza', 'provincia_residenza'),
    ('codice_provincia_erogazione', 'provincia_erogazione'),
    ('codice_regione_residenza', 'regione_residenza'),
    ('codice_asl_residenza', 'asl_residenza'),
    ('codice_comune_residenza', 'comune_residenza'),
    ('codice_descrizione_attivita', 'descrizione_attivita'),
    ('codice_regione_erogazione', 'regione_erogazione'),
    ('codice_asl_erogazione', 'asl_erogazione'),
    ('codice_struttura_erogazione', 'struttura_erogazione'),
    ('codice_tipologia_struttura_erogazione', 'tipologia_struttura_erogazione'),
    ('codice_tipologia_professionista_sanitario', 'tipologia_professionista_sanitario')
]

In [4]:
def print_details_corrections (df, code, description, code_groups, description_groups):
    
    not_unique = False

    for cod, desc_count in code_groups.items():
        if desc_count > 1:
            associated_descriptions = df[df[code] == cod][description].unique()
            print(f"The {cod} code is associated with {desc_count} descriptions: {associated_descriptions}")
            not_unique = True
            
        
    for desc, code_count in description_groups.items():
        if code_count > 1:
            associated_codes =df[df[description] == desc][code].unique()
            print(f"The {desc} description is associated with {code_count} codes: {associated_codes}")
            not_unique = True

    if not_unique:
        print(f"--> NOT unique correlation between {code} and {description}\n")



In [5]:
def remove_columns_with_unique_correlation(df, columns_pairs) -> pd.DataFrame:
    pairs_removed = []

    for code, description in columns_pairs:
        if code in df.columns and description in df.columns:
            code_groups = df.groupby(code)[description].nunique()
            description_groups = df.groupby(description)[code].nunique()

            print_details_corrections(df, code, description, code_groups, description_groups)

            unique_correlation_code_description = all(code_groups <= 1)
            unique_correlation_description_code = all(description_groups <= 1)

            if unique_correlation_code_description and unique_correlation_description_code:
                df.drop(columns=[code], inplace=True)
                print(f'Unique correlation between {code} and {description}. Column {code} removed.')
                pairs_removed.append((code, description))
        else:
            print(f'Columns {code} or {description} not found in the dataframe.')
            pairs_removed.append((code, description))

    # Update the list of columns pairs removing the ones that have been removed
    columns_pairs_updated = [pair for pair in columns_pairs if pair not in pairs_removed]
    return columns_pairs_updated

            

In [6]:
print("------------ check unique correlation between codes and descriptions: ------------\n")
columns_pairs_updated = remove_columns_with_unique_correlation(df, columns_pairs)

------------ check unique correlation between codes and descriptions: ------------

Unique correlation between codice_provincia_residenza and provincia_residenza. Column codice_provincia_residenza removed.
Unique correlation between codice_provincia_erogazione and provincia_erogazione. Column codice_provincia_erogazione removed.
Unique correlation between codice_regione_residenza and regione_residenza. Column codice_regione_residenza removed.
Unique correlation between codice_asl_residenza and asl_residenza. Column codice_asl_residenza removed.
Unique correlation between codice_comune_residenza and comune_residenza. Column codice_comune_residenza removed.
Unique correlation between codice_descrizione_attivita and descrizione_attivita. Column codice_descrizione_attivita removed.
Unique correlation between codice_regione_erogazione and regione_erogazione. Column codice_regione_erogazione removed.
Unique correlation between codice_asl_erogazione and asl_erogazione. Column codice_asl_eroga

In [7]:
def clean_codice_struttura_erogazione(df, column = 'codice_struttura_erogazione'):
    df[column] = df[column].astype(int)
    return df

In [8]:
print(df[df['struttura_erogazione'] == 'PRESIDIO OSPEDALERO CENTRALE TARANTO'].codice_struttura_erogazione.sort_values(ascending=True).unique())


df = clean_codice_struttura_erogazione(df)

print('after')
print(df[df['struttura_erogazione'] == 'PRESIDIO OSPEDALERO CENTRALE TARANTO'].codice_struttura_erogazione.sort_values(ascending=True).unique())

[160172.01 160172.03 160172.04]
after
[160172]


In [10]:
print("------------ check unique correlation between codes and descriptions with cleared codice_struttura: ------------\n")
columns_pairs_updated = remove_columns_with_unique_correlation(df, columns_pairs_updated)

------------ check unique correlation between codes and descriptions with cleared codice_struttura: ------------

The PRESIDIO OSPEDALIERO UNIFICATO description is associated with 2 codes: [ 70001 100803]
--> NOT unique correlation between codice_struttura_erogazione and struttura_erogazione



In [18]:
columns_pairs_updated

[('codice_struttura_erogazione', 'struttura_erogazione')]

In [14]:
df[df['struttura_erogazione'] == 'PRESIDIO OSPEDALIERO UNIFICATO']['codice_struttura_erogazione'].value_counts()

codice_struttura_erogazione
70001     1665
100803      60
Name: count, dtype: int64

In [15]:
def remove_data_disdetta(df) -> pd.DataFrame:
    df.drop(columns=['data_disdetta'], inplace=True)
    return df

In [16]:
df.columns

Index(['id_prenotazione', 'id_paziente', 'data_nascita', 'sesso',
       'regione_residenza', 'asl_residenza', 'provincia_residenza',
       'comune_residenza', 'tipologia_servizio', 'descrizione_attivita',
       'data_contatto', 'regione_erogazione', 'asl_erogazione',
       'provincia_erogazione', 'struttura_erogazione',
       'codice_struttura_erogazione', 'tipologia_struttura_erogazione',
       'id_professionista_sanitario', 'tipologia_professionista_sanitario',
       'data_erogazione', 'ora_inizio_erogazione', 'ora_fine_erogazione',
       'data_disdetta'],
      dtype='object')

In [17]:
remove_data_disdetta(df)
df.columns

Index(['id_prenotazione', 'id_paziente', 'data_nascita', 'sesso',
       'regione_residenza', 'asl_residenza', 'provincia_residenza',
       'comune_residenza', 'tipologia_servizio', 'descrizione_attivita',
       'data_contatto', 'regione_erogazione', 'asl_erogazione',
       'provincia_erogazione', 'struttura_erogazione',
       'codice_struttura_erogazione', 'tipologia_struttura_erogazione',
       'id_professionista_sanitario', 'tipologia_professionista_sanitario',
       'data_erogazione', 'ora_inizio_erogazione', 'ora_fine_erogazione'],
      dtype='object')